# ***PS3***

In [5]:
# import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

In [3]:
# grab the data
filepaths = {
    "x_train": "./trainX.txt",
    "y_train": "./trainY.txt",
    "x_test": "./testX.txt",
    "y_test": "./testY.txt"
}
x_train = pd.read_csv(filepaths["x_train"])
y_train = pd.read_csv(filepaths["y_train"])
x_test = pd.read_csv(filepaths["x_test"])
y_test = pd.read_csv(filepaths["y_test"])

## 1) **Bayesian Logistic Regression for Image Classification**

## 2) **PCA**

In [18]:
# pca projection function for a given number of components
def pca_projection(dataset_x, components):
  pca = PCA(n_components = components)

  return pca.fit_transform(X = dataset_x)

# resulting projections
pca_projections_dict = {
    "10_pc": pca_projection(x_train, 10),
    "20_pc": pca_projection(x_train, 20),
    "30_pc": pca_projection(x_train, 30)
}

## 3) **Augmentation**

## **Sources**
scikit-learn.org
